# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [2]:
import pandas as pd
import numpy as np
from scipy import stats

In [3]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [7]:
df = data[['race','call']]
df.race.value_counts()

b    2435
w    2435
Name: race, dtype: int64

We know there are 4870 rows. No we see that both categories have the same size. Next we can figure out how many callbacks were received, per race.

In [8]:
df.groupby(['race','call']).size().unstack()

call,0.0,1.0
race,,
b,2278,157
w,2200,235


So we can see that the number of callbacks are not the same, even though the initial sizes were the same. So we want to quantify whether this difference is statistical significant. 

1. What test is appropriate for this problem? Does CLT apply?
The data is independent and identically distributed, as defined in the problem statement. Also there is much more data than 30 samples. Therefore, CLT does apply. Given that we want to compare two sample proportions (and not means) with known variances the best test is the z-test. This will be a two-sided test.

2. What are the null and alternate hypotheses?
The null hypothesis is that the difference between the rate of callbacks depending on race is zero. The alternate hypothesis then presumes that the difference is non-zero.

3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [12]:
w = data[data.race=='w']
b = data[data.race=='b']

235.0

In [15]:
# Your solution to Q3 here
df.groupby(['race','call']).size().apply(lambda x: x/2435.).unstack()

#Calculate the number of callbacks for each race
num_call_w = sum(w.call)
num_call_b = sum(b.call)

#Calculate the total rows for each race
num_tot_w = len(w.call)
num_tot_b = len(b.call)

#Calculate the proportion of callback for each race
prop_w = num_call_w / num_tot_w
prop_b = num_call_b / num_tot_b

#Calculate the pooled proportion
prop_pool = (prop_w * num_tot_w + prop_b * num_tot_b) / (num_tot_w + num_tot_b)

#Calculate the difference in proportions
prop_diff = np.abs(prop_w-prop_b)

#Calculate the standard error between proportions
std_error = np.sqrt(prop_pool * (1-prop_pool) * ((1./num_tot_w) + (1./num_tot_b)))

#Calculate the z-score for this exercise
z = prop_diff / std_error

#Assume a confidence level of 0.95
CL = 0.95

#Calculate the margin of error
margin_error = stats.norm.ppf(1-((1-CL)/2)) * std_error

#Calculate the 95% CI from the difference of proportions
ci_lower, ci_upper = stats.norm.interval(CL, loc=prop_diff, scale=std_error)

#Calculate the p-value 
p_value = (1-stats.norm.cdf(z))*2

print("  %-30s | %s" % ('Calculation', 'Value'))
print(60 * '=')
print("  %-30s | %.8f" % ('Callback proportion for W', prop_w))
print("  %-30s | %.8f" % ('Callback proportion for B', prop_b))
print("  %-30s | %.8f" % ('Difference in proportions', prop_diff))
print("  %-30s | %.8f" % ('Pooled proportions', prop_pool))
print("  %-30s | %.8f" % ('Standard Error', std_error))
print("  %-30s | %.8f" % ('z-test statistic', z))
print("  %-30s | %.8f" % ('Margin of Error', margin_error))
print("  %-30s | %.8f to %.8f" % ('%i%% Confidence Interval' % (CL*100), ci_lower, ci_upper))
print("  %-30s | %.8f" % ('z-test p-value', p_value))

  Calculation                    | Value
  Callback proportion for W      | 0.09650924
  Callback proportion for B      | 0.06447639
  Difference in proportions      | 0.03203285
  Pooled proportions             | 0.08049281
  Standard Error                 | 0.00779689
  z-test statistic               | 4.10841215
  Margin of Error                | 0.01528163
  95% Confidence Interval        | 0.01675122 to 0.04731449
  z-test p-value                 | 0.00003984


Last 3 ros of the table show margin of erro, confidence interval and p value.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

4. Write a story describing the statistical significance in the context or the original problem.
The p-value from the z-test is very small. The CI also supports this p-value since it does not include zero (remember its calculated for the difference in proportions). This means the difference between the two proportions is not zero and we can reject the null hypothesis. In a vacuum, this analysis would indicate that indeed there is a bias in who gets a callback for an employment application.

5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
There are other factors that are likely to play in this scenario. We do not know how much name (which is used as a proxy for race, another complication) actually affects employment applications. Many of these other factors are likely present in the given dataset, as they include info on sex, education, and background. Without looking into these other features of the dataset it would be difficult to say that race (or name) is the most important factor in callback success. A better analysis would use regression/machine learning to map and learn the relationships between all the relavant factors.